In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
from utils import *

In [3]:
df = load()
df.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay,Airline_DelayRate,Route,Route_AvgDelay
0,0,CO,269,SFO,IAH,3,15,205,1,0.566199,SFO_IAH,0.800866
1,1,US,1558,PHX,CLT,3,15,222,1,0.335971,PHX_CLT,0.418972
2,2,AA,2400,LAX,DFW,3,20,165,1,0.388470,LAX_DFW,0.347426
3,3,AA,2466,SFO,DFW,3,20,195,1,0.388470,SFO_DFW,0.525836
4,4,AS,108,ANC,SEA,3,30,202,0,0.339290,ANC_SEA,0.344519


In [27]:
print("Airlines: ")
print(df['Airline'].unique())

# print(df['AirportFrom'].unique().sort() == df['AirportTo'].unique().sort())
# -> True

print("All possible Airports: ")
print(df['AirportTo'].unique())

Airlines: 
['CO' 'US' 'AA' 'AS' 'DL' 'B6' 'HA' 'OO' '9E' 'OH' 'EV' 'XE' 'YV' 'UA'
 'MQ' 'FL' 'F9' 'WN']
All possible Airports: 
['IAH' 'CLT' 'DFW' 'SEA' 'MSP' 'DTW' 'ORD' 'ATL' 'PDX' 'JFK' 'SLC' 'HNL'
 'PHX' 'MCO' 'OGG' 'LAX' 'KOA' 'ITO' 'SFO' 'MIA' 'IAD' 'SMF' 'PHL' 'LIH'
 'DEN' 'LGA' 'MEM' 'CVG' 'YUM' 'CWA' 'MKE' 'BQN' 'FAI' 'LAS' 'ANC' 'BOS'
 'LGB' 'FLL' 'SJU' 'EWR' 'DCA' 'BWI' 'RDU' 'MCI' 'TYS' 'SAN' 'ONT' 'OAK'
 'MDW' 'BNA' 'DAL' 'CLE' 'JAX' 'JNU' 'RNO' 'ELP' 'SAT' 'OTZ' 'MBS' 'BDL'
 'STL' 'HOU' 'AUS' 'SNA' 'SJC' 'LIT' 'TUS' 'TUL' 'CMH' 'LAN' 'IND' 'AMA'
 'CRP' 'PIT' 'RKS' 'FWA' 'TPA' 'PBI' 'JAN' 'DSM' 'ADQ' 'GRB' 'PVD' 'ABQ'
 'SDF' 'RSW' 'MSY' 'BUR' 'BOI' 'TLH' 'BHM' 'ACV' 'ORF' 'BET' 'KTN' 'RIC'
 'SRQ' 'BTR' 'XNA' 'MHT' 'GRR' 'SBN' 'SBA' 'ROA' 'CID' 'GPT' 'MFR' 'SGU'
 'HPN' 'OMA' 'OTH' 'GSP' 'LMT' 'BUF' 'MSN' 'BFL' 'CAE' 'HRL' 'OKC' 'SYR'
 'COS' 'BTV' 'CDC' 'SCC' 'DAY' 'SJT' 'TVC' 'ROC' 'ISP' 'MRY' 'SBP' 'MLI'
 'MOB' 'CIC' 'SAV' 'FAT' 'EKO' 'GEG' 'ECP' 'LFT' 'SUN' 'HSV' 'SHV' 'C

Percentage of flights delayed for {airline} on route {from} to {to}

In [15]:
route_delay = (
    df.groupby(['Airline', 'AirportFrom', 'AirportTo'])
    .agg(
        total_flights = ('id', 'count'),
        delayed_flights = ('Delay', 'sum')
    )
    .assign(delay_rate=lambda x: x['delayed_flights'] / x['total_flights'])
    .reset_index()
)
route_delay.head()

,Airline,AirportFrom,AirportTo,total_flights,delayed_flights,delay_rate
0,9E,ABE,DTW,85,40,0.470588
1,9E,ABR,MSP,2,1,0.500000
2,9E,ALB,ATL,41,9,0.219512
3,9E,ALB,DTW,90,39,0.433333
4,9E,ALB,JFK,31,3,0.096774


In [17]:
route_delay.to_csv("../webapp/precalc_metrics/route_airline_delay_rate.csv", index=False)

Delay Rate for Route regardless of Airline

In [19]:
route_only_delay = (
    df.groupby(['AirportFrom', 'AirportTo'])
      .agg(
          total_flights=('id', 'count'),
          delayed_flights=('Delay', 'sum')
      )
      .assign(delay_rate=lambda x: x['delayed_flights'] / x['total_flights'])
      .reset_index()
)
route_only_delay.head()

,AirportFrom,AirportTo,total_flights,delayed_flights,delay_rate
0,ABE,ATL,31,7,0.225806
1,ABE,CLT,31,2,0.064516
2,ABE,DTW,85,40,0.470588
3,ABE,FLL,17,4,0.235294
4,ABE,MCO,31,9,0.290323


In [20]:
route_only_delay.to_csv('../webapp/precalc_metrics/route_delay_rate.csv', index=False)

Airline Delay Rate

In [21]:
airline_delay = (
    df.groupby('Airline')
      .agg(
          total_flights=('id', 'count'),
          delayed_flights=('Delay', 'sum')
      )
      .assign(delay_rate=lambda x: x['delayed_flights'] / x['total_flights'])
      .reset_index()
)
airline_delay.head()

,Airline,total_flights,delayed_flights,delay_rate
0,9E,20686,8226,0.397660
1,AA,45656,17736,0.388470
2,AS,11471,3892,0.339290
3,B6,18112,8459,0.467038
4,CO,21118,11957,0.566199


In [22]:
airline_delay.to_csv("../webapp/precalc_metrics/airline_delay_rate.csv", index=False)

In [28]:
df['Flight'].value_counts()

Flight
16      420
5       407
9       401
8       396
62      364
       ... 
5131      1
5559      1
3349      1
3413      1
3184      1
Name: count, Length: 6585, dtype: int64